<a href="https://www.kaggle.com/code/akscent/fitmlforcharn-prediction?scriptVersionId=152319747" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [125]:
import numpy as np
import pandas as pd
import seaborn as sns

from catboost import CatBoostClassifier
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from pprint import pprint
from warnings import filterwarnings

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

filterwarnings("ignore", category=pd.errors.PerformanceWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=Warning)
filterwarnings("ignore", category=DeprecationWarning)



In [2]:
# Check if GPU is available
try:
    import cupy as cp
    gpu_available = True
except ImportError:
    gpu_available = False

# Set n_jobs parameter for classifiers that support it
n_jobs = -1 if gpu_available else 1

In [3]:
import wandb
#https://wandb.ai/authorize
wandb.login()

wandb: Currently logged in as: irinyakov2016. Use `wandb login --relogin` to force relogin


True

In [4]:
CONFIG = dict ()

CONFIG['model_name'] = 'DJStacking'
print('Training configuration: ', CONFIG)

# Initialize W&B run
run = wandb.init(project='AlfaSibHack',
                 config=CONFIG,
                 group='DJStacking', 
                 job_type='train')


Training configuration:  {'model_name': 'DJStacking'}


# Load data

In [129]:
# train_df = pd.read_parquet('/kaggle/input/sibalfahack/Siberian Alfa Hack Materials/train.parquet')
# test_df = pd.read_parquet('/kaggle/input/sibalfahack/Siberian Alfa Hack Materials/test.parquet')

x_train = pd.read_csv('/kaggle/input/sibalfahack/X_train_v1.csv')
y_train = pd.read_csv('/kaggle/input/sibalfahack/y_train_v1.csv')
test_df = pd.read_csv('/kaggle/input/sibalfahack/test_v1 (2).csv')
y_cols = ["target"]
data = x_train
data[y_cols] = y_train

In [130]:
# num_cols = [
#        'sum_b_oper_3m', 'cnt_b_oper_3m', 'sum_c_oper_3m', 'cnt_c_oper_3m',
#        'sum_deb_d_oper_3m', 'cnt_deb_d_oper_3m', 'sum_cred_d_oper_3m',
#        'cnt_cred_d_oper_3m', 'sum_deb_e_oper_3m', 'cnt_deb_e_oper_3m',
#        'cnt_days_deb_e_oper_3m', 'sum_cred_e_oper_3m', 'cnt_cred_e_oper_3m',
#        'cnt_days_cred_e_oper_3m', 'sum_deb_f_oper_3m', 'cnt_deb_f_oper_3m',
#        'cnt_days_deb_f_oper_3m', 'sum_cred_f_oper_3m', 'cnt_cred_f_oper_3m',
#        'cnt_days_cred_f_oper_3m', 'sum_deb_g_oper_3m', 'cnt_deb_g_oper_3m',
#        'cnt_days_deb_g_oper_3m', 'sum_cred_g_oper_3m', 'cnt_cred_g_oper_3m',
#        'cnt_days_cred_g_oper_3m', 'sum_deb_h_oper_3m', 'cnt_deb_h_oper_3m',
#        'cnt_days_deb_h_oper_3m', 'sum_cred_h_oper_3m', 'cnt_cred_h_oper_3m',
#        'cnt_days_cred_h_oper_3m'
# ]

# cat_cols = [
#     "Sex",
#     "IsSeniorCitizen",
#     "HasPartner",
#     "HasChild",
#     "HasPhoneService",
#     "HasMultiplePhoneNumbers",
#     "HasInternetService",
#     "HasOnlineSecurityService",
#     "HasOnlineBackup",
#     "HasDeviceProtection",
#     "HasTechSupportAccess",
#     "HasOnlineTV",
#     "HasMovieSubscription",
#     "HasContractPhone",
#     "IsBillingPaperless",
#     "PaymentMethod",
# ]

# target = 'total_target'

In [131]:
# train_df = pd.concat([train_df[num_cols], train_df[target]], axis = 1)
# train_df

In [132]:
# train_df.dropna(inplace=True)

# Choose model

* Используем choos model из предыдущей работы - расширенный - https://www.kaggle.com/code/akscent/choice-of-boosting
* Отличный ноутбук с оценкой и ансамблированием моделей - https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy
* Далее оптимизаторы гиперпараметр**ов - https://www.kaggle.com/code/akscent/ods-boosting
* Далее стекинг - https://www.kaggle.com/code/akscent/ods-boosting ; https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/ ; https://github.com/a-milenkin/Competitive_Data_Science/blob/main/notebooks/6.3%20-%20Automatic%20Stacking.ipynb
* приемы в мл - https://github.com/Dyakonov/ml_hacks/tree/master

In [133]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, train

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from tqdm.notebook import tqdm

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline

In [134]:
# df_target_0 = train_df[train_df['total_target'] == 0].head(25000)
# df_target_1 = train_df[train_df['total_target'] == 1].head(25000)

# train_df = pd.concat([df_target_0, df_target_1], ignore_index=True)
# train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# One-Hot-Encoder for city type

In [135]:
test_df.isnull().sum().max()

0

In [136]:
object_columns = x_train.select_dtypes(include=['object']).columns
x_train[object_columns] = x_train[object_columns].astype('category')
test_df[object_columns] = test_df[object_columns].astype('category')

In [137]:
cat_cols = [
    'channel_code',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment', 'city_type_1252', 'city_type_3597', 'city_type_other'
]
type_city = ['city_type_1252', 'city_type_3597', 'city_type_other']

In [29]:
x_train.columns

Index(['rko_start_months', 'balance_amt_avg', 'balance_amt_max',
       'balance_amt_min', 'balance_amt_day_avg', 'channel_code',
       'index_city_code', 'ogrn_days_end_month', 'ogrn_days_end_quarter',
       'ogrn_month', 'ogrn_year', 'branch_code', 'ft_registration_date',
       'max_founderpres', 'min_founderpres', 'ogrn_exist_months', 'okved',
       'segment', 'sum_of_paym_2m', 'sum_of_paym_6m', 'sum_of_paym_1y',
       'sum_a_oper_1m', 'cnt_a_oper_1m', 'sum_b_oper_1m', 'cnt_b_oper_1m',
       'sum_c_oper_1m', 'cnt_c_oper_1m', 'sum_deb_d_oper_1m',
       'cnt_deb_d_oper_1m', 'sum_cred_d_oper_1m', 'cnt_cred_d_oper_1m',
       'sum_deb_e_oper_1m', 'cnt_deb_e_oper_1m', 'cnt_days_deb_e_oper_1m',
       'sum_cred_e_oper_1m', 'cnt_cred_e_oper_1m', 'cnt_days_cred_e_oper_1m',
       'sum_deb_f_oper_1m', 'cnt_deb_f_oper_1m', 'cnt_days_deb_f_oper_1m',
       'sum_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'cnt_days_cred_f_oper_1m',
       'sum_deb_g_oper_1m', 'cnt_deb_g_oper_1m', 'cnt_days_de

In [138]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ["city_type"]

encoder = OneHotEncoder(drop='first', sparse=False)
X_copy_encoded = pd.DataFrame(encoder.fit_transform(x_train[categorical_cols]))
new_columns = [
    f"{col}_{category}" 
    for col, categories in zip(categorical_cols, encoder.categories_) 
    for category in categories[1:]
]

X_copy_encoded.columns = new_columns
x_train = pd.concat([x_train.drop(columns=categorical_cols), X_copy_encoded], axis=1)
print(x_train.head())


X_copy_encoded = pd.DataFrame(encoder.fit_transform(test_df[categorical_cols]))
new_columns = [
    f"{col}_{category}" 
    for col, categories in zip(categorical_cols, encoder.categories_) 
    for category in categories[1:]
]

X_copy_encoded.columns = new_columns
test_df = pd.concat([test_df.drop(columns=categorical_cols), X_copy_encoded], axis=1)
test_df.head()

   rko_start_months  balance_amt_avg  balance_amt_max  balance_amt_min  \
0         48.871217     27046.725915     73020.902425       256.536946   
1         19.530576     54167.143820    130950.622425      2928.804640   
2          9.706201    292152.446430    827241.812425     39123.394462   
3         92.569902    142491.809620    281227.112425     11816.038314   
4         26.538856     72832.525309    119232.722425     45824.510960   

   balance_amt_day_avg  channel_code  index_city_code  ogrn_days_end_month  \
0         33120.446473            29               -1             8.328600   
1         60240.717448             4               -1            12.051663   
2        298226.317987            26               -1            31.138984   
3        148566.636207            30               -1             4.486545   
4         78907.305464            26               -1            10.785315   

   ogrn_days_end_quarter  ogrn_month  ...  sum_deb_h_oper_3m  \
0              78.7986

,rko_start_months,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,city_type_1252,city_type_3597,city_type_other
0,61.061808,2.735668e+04,7.416109e+04,255.723506,3.343082e+04,34,-1,12.170908,52.804632,9,...,4.257042,33195.968822,1.229488,0.359263,301442.064297,7.049883,7.069157,0.0,0.0,0.0
1,16.803743,1.654578e+06,1.729187e+06,2265.723506,1.660652e+06,37,212,22.170908,62.804632,2,...,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157,0.0,1.0,0.0
2,15.448904,5.315305e+04,9.921213e+04,26294.763506,5.922719e+04,7,-1,19.170908,90.804632,11,...,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157,0.0,0.0,0.0
3,11.287614,7.648909e+04,1.224288e+05,49710.403506,8.256323e+04,32,-1,10.170908,81.804632,4,...,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157,0.0,0.0,0.0
4,7.997291,9.232664e+04,1.755382e+05,255.723506,9.840078e+04,37,31,31.170908,41.804632,10,...,0.257042,135758.248681,2.229488,1.359263,126742.064297,2.049883,2.069157,0.0,1.0,0.0


In [140]:
x_cols = list(x_train.drop(columns = y_cols).columns)
test_df = test_df[x_cols]
x_cols

['rko_start_months',
 'balance_amt_avg',
 'balance_amt_max',
 'balance_amt_min',
 'balance_amt_day_avg',
 'channel_code',
 'index_city_code',
 'ogrn_days_end_month',
 'ogrn_days_end_quarter',
 'ogrn_month',
 'ogrn_year',
 'branch_code',
 'ft_registration_date',
 'max_founderpres',
 'min_founderpres',
 'ogrn_exist_months',
 'okved',
 'segment',
 'sum_of_paym_2m',
 'sum_of_paym_6m',
 'sum_of_paym_1y',
 'sum_a_oper_1m',
 'cnt_a_oper_1m',
 'sum_b_oper_1m',
 'cnt_b_oper_1m',
 'sum_c_oper_1m',
 'cnt_c_oper_1m',
 'sum_deb_d_oper_1m',
 'cnt_deb_d_oper_1m',
 'sum_cred_d_oper_1m',
 'cnt_cred_d_oper_1m',
 'sum_deb_e_oper_1m',
 'cnt_deb_e_oper_1m',
 'cnt_days_deb_e_oper_1m',
 'sum_cred_e_oper_1m',
 'cnt_cred_e_oper_1m',
 'cnt_days_cred_e_oper_1m',
 'sum_deb_f_oper_1m',
 'cnt_deb_f_oper_1m',
 'cnt_days_deb_f_oper_1m',
 'sum_cred_f_oper_1m',
 'cnt_cred_f_oper_1m',
 'cnt_days_cred_f_oper_1m',
 'sum_deb_g_oper_1m',
 'cnt_deb_g_oper_1m',
 'cnt_days_deb_g_oper_1m',
 'sum_cred_g_oper_1m',
 'cnt_cred_g_op

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_classification_metrics(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    print(f"Classification Report for {model_name}:\n", classification_report(y_true, y_pred))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [71]:
MLA = [
    # Ensemble Methods
#     ensemble.AdaBoostClassifier(),
    ensemble.ExtraTreesClassifier(n_jobs=n_jobs, verbose = 0),

    # GLM
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(n_jobs=n_jobs),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(n_jobs=n_jobs),
#     linear_model.Perceptron(n_jobs=n_jobs),

    # Navies Bayes
#     naive_bayes.BernoulliNB(),
#     naive_bayes.GaussianNB(),

    # Nearest Neighbor
#     neighbors.KNeighborsClassifier(n_jobs=n_jobs),

    # SVM
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
#     svm.LinearSVC(),

    # Trees
#     tree.DecisionTreeClassifier(),
#     tree.ExtraTreeClassifier(),

    # Discriminant Analysis
#     discriminant_analysis.LinearDiscriminantAnalysis(),
#     discriminant_analysis.QuadraticDiscriminantAnalysis(),

    # xgboost
    XGBClassifier(tree_method='gpu_hist') if gpu_available else XGBClassifier(verbose = -1),

    # LightGBM
    LGBMClassifier(device_type='gpu') if gpu_available else LGBMClassifier(verbose = -1),

    # CatBoost
    CatBoostClassifier(task_type='GPU') if gpu_available else CatBoostClassifier(verbose = 0)
]

def evaluate_ml_algorithms(MLA, data, data_x, target, cv_splits=5, test_size=0.1, train_size=0.4, random_state=43):
    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
    MLA_compare = pd.DataFrame(columns=MLA_columns)
    
    MLA_predict = data[target]
    
    cv_split = model_selection.StratifiedShuffleSplit(n_splits=cv_splits, test_size=test_size, train_size=train_size, random_state=random_state)

    row_index = 0
    for alg in  tqdm(MLA, total=len(MLA), desc="ML first estimate"):
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        
        cv_results = model_selection.cross_validate(alg, data[data_x], data[target], cv=cv_split, return_train_score = True)
        print(f"{MLA_name} estimated: {cv_results['test_score'].mean()}")
        MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
        MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std() * 3
        alg.fit(data[data_x], data[target])
        MLA_predict[MLA_name] = alg.predict(data[data_x])
        evaluate_classification_metrics(MLA_predict[MLA_name], data[target], MLA_name)
        
        row_index += 1

    MLA_compare.sort_values(by=['MLA Test Accuracy Mean'], ascending=False, inplace=True)
    
    return MLA_compare

result = evaluate_ml_algorithms(MLA, x_train, x_cols, y_cols)

sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = result)
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')

In [ ]:
result["MLA Test Accuracy Mean"] = result["MLA Test Accuracy Mean"].astype(float)
list_best_models = list(result.nlargest(3, 'MLA Test Accuracy Mean')["MLA Name"])

In [84]:
list_best_models = ['CatBoostClassifier',
 'ExtraTreesClassifier',
 'LGBMClassifier',
 'XGBClassifier']

# Search hyperparams

In [24]:
x_train.select_dtypes(include=['category']).columns

Index([], dtype='object')

In [57]:
x_train[x_cols]

,rko_start_months,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,index_city_code,ogrn_days_end_month,ogrn_days_end_quarter,ogrn_month,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,city_type_1252,city_type_3597,city_type_other
0,48.871217,2.704673e+04,7.302090e+04,2.565369e+02,3.312045e+04,29,-1,8.328600,78.798676,3,...,0.064993,3.319602e+04,1.229488,0.396969,3.235240e+05,12.420855,9.069157,0.0,0.0,0.0
1,19.530576,5.416714e+04,1.309506e+05,2.928805e+03,6.024072e+04,4,-1,12.051663,83.783868,4,...,0.270542,3.319549e+04,1.229488,-0.137339,1.456416e+05,2.124925,2.069157,0.0,0.0,1.0
2,9.706201,2.921524e+05,8.272418e+05,3.912339e+04,2.982263e+05,26,-1,31.138984,42.134142,6,...,23.431839,2.650712e+05,4.229488,3.595162,1.257271e+05,0.999583,1.069157,0.0,0.0,0.0
3,92.569902,1.424918e+05,2.812271e+05,1.181604e+04,1.485666e+05,30,-1,4.486545,44.670808,9,...,0.990581,4.950943e+05,9.229488,8.582252,3.703923e+05,3.066714,3.069157,0.0,0.0,0.0
4,26.538856,7.283253e+04,1.192327e+05,4.582451e+04,7.890731e+04,26,-1,10.785315,82.925218,8,...,0.393927,3.319632e+04,1.229488,0.276606,1.257268e+05,1.219935,1.069157,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329995,-999.000000,1.324653e+06,1.536300e+06,8.965601e+04,1.330727e+06,31,224,28.718252,39.917222,1,...,0.177506,2.038701e+06,51.229488,22.182922,2.006430e+05,3.963913,2.069157,0.0,0.0,1.0
329996,-999.000000,2.703910e+04,7.307309e+04,2.555530e+02,3.311267e+04,7,-1,20.329021,63.170979,0,...,0.301299,3.319599e+04,1.229488,0.225770,1.257279e+05,1.114018,1.069157,0.0,1.0,0.0
329997,-999.000000,1.881385e+07,2.051423e+07,1.634543e+07,1.881992e+07,34,71,10.267192,21.045848,1,...,0.150675,2.151680e+05,16.229488,8.413681,1.309472e+07,5.397099,5.069157,0.0,1.0,0.0
329998,-999.000000,5.068857e+05,1.580485e+06,5.915166e+04,5.129597e+05,10,58,24.820807,35.942701,6,...,53.062204,1.874074e+07,353.229488,58.237079,1.078043e+07,294.991216,67.069157,0.0,1.0,0.0


In [ ]:
#     # Список гиперпараметров для каждой модели
#     MLA_params = [
#         {'name': 'AdaBoost', 'params': {'n_estimators': trial.suggest_int('n_estimators', 2000, 5000), 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),}},
#     #     {'name': 'BaggingClassifier', 'params': {'n_estimators': [10, 50, 100], 'max_samples': [0.5, 0.7, 1.0]}},
#     #     {'name': 'ExtraTrees', 'params': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2']}},
#     #     {'name': 'GradientBoosting', 'params': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]}},
#     #     {'name': 'RandomForest', 'params': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2']}},
#     #     {'name': 'LogisticRegressionCV', 'params': {'Cs': [1, 10, 100], 'penalty': ['l1', 'l2']}},
#     #     {'name': 'PassiveAggressiveClassifier', 'params': {'C': [0.1, 1, 10], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'RidgeClassifierCV', 'params': {'alphas': [(1e-3, 1e-2, 1e-1), (1, 10, 100)]}},
#     #     {'name': 'SGDClassifier', 'params': {'alpha': [1e-3, 1e-2, 1e-1], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'Perceptron', 'params': {'alpha': [1e-3, 1e-2, 1e-1], 'max_iter': [100, 200, 500]}},
#     #     {'name': 'BernoulliNB', 'params': {'alpha': [0.1, 1, 2], 'binarize': [0.0, 0.5, 1.0]}},
#     #     {'name': 'GaussianNB', 'params': {}},
#         {'name': 'KNeighborsClassifier', 'params': {'n_neighbors': trial.suggest_int('n_neighbors',3, 7), 
#                                                     'weights': trial.suggest_categorical("weights", ['uniform', 'distance'])}},
#     #     {'name': 'SVC', 'params': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}},
#     #     {'name': 'NuSVC', 'params': {'nu': [0.1, 0.3, 0.5], 'kernel': ['linear', 'rbf']}},
#     #     {'name': 'LinearSVC', 'params': {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}},
#     #     {'name': 'DecisionTreeClassifier', 'params': {'criterion': ['gini', 'entropy'], 'max_depth': [3, 5, 7]}},
#         {'name': 'ExtraTreesClassifier', 'params': {'criterion': trial.suggest_categorical("criterion", ['gini', 'entropy']), 
#                                                    'max_depth': trial.suggest_categorical('max_depth', [3, 7])}},

#     #     {'name': 'LinearDiscriminantAnalysis', 'params': {}},
#     #     {'name': 'QuadraticDiscriminantAnalysis', 'params': {}},
#         {'name': 'XGBClassifier', 'params': {
#             'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#             'n_estimators': trial.suggest_int('n_estimators', 300, 500),
#             'eta': trial.suggest_float('eta', 0.01, 0.1),
#             'booster': trial.suggest_categorical("booster", ['gbdt', 'dart', 'goss']),
#             'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
#             'objective': 'binary:logistic',}},
#         {'name': 'CatBoostClassifier', 'params': {
#             "iterations": trial.suggest_int("iterations", 500, 800),
#             "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
#             "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
#             "depth": trial.suggest_int("depth", 3, 6),
#             'loss_function':'Logloss',
#             'use_best_model': True,
#             'nan_mode': trial.suggest_categorical("nan_mode", ["Min", "Max"]),
#         }},
#         {'name': 'LGBMClassifier', 'params': {'num_iterations': trial.suggest_int('num_iterations', 300, 500),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
#             'num_leaves': trial.suggest_int('num_leaves', 10, 20),
#             'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5),
#             'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss']),
#             'objective': 'binary',
#             'metric': 'binary_logloss',
#             'force_col_wise': True,
#             'is_unbalance': True}}
#     ]
    
#     available_model_names = [model['name'] for model in MLA_params]
#     if model_name not in available_model_names:
#         print(f"Error: Model name '{model_name}' not found in available models. Please check the model name.")
#     else:
#         filtered_MLA_params = None
#         for model in MLA_params:
#             if model['name'] == model_name:
#                 filtered_MLA_params = model['params']
#                 break
#             else:
#                 continue

#         if not filtered_MLA_params:
#             print(f"Warning: No parameters found for the specified model '{model_name}'.")
#         else:
#             if not filtered_MLA_params:
#                 print(f"Model {model_name} not found in MLA_params. Skipping optimization.")
#                 return 0

In [106]:
import optuna
import torch
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit

def fit_MLA(trial, X_train, y_train, X_valid, y_valid, MLA, model_name):
            
    if model_name == "CatBoostClassifier":
        params = {
            "iterations": trial.suggest_int("iterations", 500, 800),
            "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.01),
            "auto_class_weights": trial.suggest_categorical("auto_class_weights", ["SqrtBalanced", "Balanced", "None"]),
            "depth": trial.suggest_int("depth", 3, 6),
            'loss_function':'Logloss',
            'use_best_model': True,
            'nan_mode': trial.suggest_categorical("nan_mode", ["Min", "Max"])}
        column_numbers = [5, 6, 9, 10, 11, 16, 17, 88, 89, 90]
        X_train[type_city] = X_train[type_city].astype('str')
        X_valid[type_city] = X_valid[type_city].astype('str')
        for col in cat_cols:
            X_train[col] = pd.Categorical(X_train[col])
            X_valid[col] = pd.Categorical(X_valid[col])
        train_dataset = Pool(data=X_train, label=y_train, cat_features = column_numbers)
        eval_dataset = Pool(data=X_valid, label=y_valid, cat_features = column_numbers)
        clf = CatBoostClassifier(verbose = 0, random_seed = 41, **params)
        clf.fit(train_dataset, eval_set=eval_dataset, early_stopping_rounds=300)
    elif model_name == "LGBMClassifier":
        params = {'num_iterations': trial.suggest_int('num_iterations', 300, 500),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
            'num_leaves': trial.suggest_int('num_leaves', 10, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5),
            'boosting_type': trial.suggest_categorical("boosting_type", ['gbdt', 'dart', 'goss']),
            'objective': 'binary',
            'metric': 'binary_logloss',
            'max_bin': trial.suggest_int('max_bin', 255, 4095),
            'force_col_wise': True,
            'is_unbalance': True}
        
        column_numbers = [5, 6, 9, 10, 11, 16, 17, 88, 89, 90]
        X_train[type_city] = X_train[type_city].astype('str')
        X_valid[type_city] = X_valid[type_city].astype('str')
        for col in cat_cols:
            X_train[col] = pd.Categorical(X_train[col])
            X_valid[col] = pd.Categorical(X_valid[col])
        clf = LGBMClassifier(cat_feature = column_numbers, verbose = -1, random_seed = 42)
        clf = clf.set_params(**params)
        clf.fit(X_train, y_train)
    elif model_name == "XGBClassifier":
        params = {
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 300, 500),
            'eta': trial.suggest_float('eta', 0.01, 0.1),
            'min_child_weight': trial.suggest_float('min_child_weight', 1, 5),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 0.5),
            'objective': 'binary:logistic',}
        clf = XGBClassifier(tree_method = 'hist', device= 'cuda') if gpu_available else XGBClassifier(verbose = -1, random_seed = 43)
        clf = clf.set_params(**params)
        clf.fit(X_train, y_train)
    else:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_float('min_samples_split', 0.1, 1.0),
            'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 0.5),
            'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
            'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample']),
            'n_classes_': 2,
            'random_state': 42}
        clf = MLA
        clf = clf.set_params(**params)
        clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_valid)[:, 1]
    return clf, y_pred

# Определение функции objective
def objective(trial, return_models=False):
    
    n_splits = 3
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2, random_state=42)
    scores, models = [], []

    for train_idx, valid_idx in sss.split(x_train[x_cols], x_train[y_cols]):
        X_train, X_valid = x_train[x_cols].iloc[train_idx, :], x_train[x_cols].iloc[valid_idx, :]
        y_train, y_valid = x_train[y_cols].iloc[train_idx], x_train[y_cols].iloc[valid_idx]

        X_train.columns = X_train.columns.str.replace(' ', '_')
        X_valid.columns = X_valid.columns.str.replace(' ', '_')

        clf, y_pred = fit_MLA(trial, X_train, y_train, X_valid, y_valid, model, model_name)
        roc_auc = roc_auc_score(y_valid, y_pred)
        scores.append(roc_auc)
        models.append(clf)
        break
        
    if return_models:
        return np.mean(scores), models
    else:
        return np.mean(scores)

# Список для сохранения результатов оптимизации по моделям
optimization_results, optimization_models = [], []

# Оптимизация гиперпараметров для каждой модели
for model in tqdm(MLA, total=len(MLA), desc='Optimizing models'):
    model_name = model.__class__.__name__
    if model_name in list_best_models:
    #     result, clm = optimize_model(x_train[x_cols], x_train[y_cols], model_name, model)
        study = optuna.create_study(direction='maximize', sampler=TPESampler())
    #     study.optimize(lambda trial: objective(trial, X, y, MLA, model_name), n_trials=5)
        study.optimize(objective, n_trials=10, n_jobs = -1, show_progress_bar=True,)
        best_model_params = study.best_params
        valid_scores = objective(optuna.trial.FixedTrial(study.best_params))

        result = {
            'model_name': model_name,
            'num_trials': len(study.trials),
            'best_trial_value': valid_scores,
            'best_trial_params': best_model_params,
        }   
        optimization_results.append(result)
        print(f'Result for {model_name}: {valid_scores}')
    else:
        continue

print("Optimization results:")
for result in optimization_results:
    print(f"Model: {result['model_name']}, Num Trials: {result['num_trials']}, Best Trial Value: {result['best_trial_value']}")
    print(f"Best Trial Params: {result['best_trial_params']}")
    print("\n")

Optimizing models:   0%|          | 0/4 [00:00<?, ?it/s]

[I 2023-11-25 21:48:24,338] A new study created in memory with name: no-name-1ed8bb05-9b58-477b-9375-ac927d42084d


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-11-25 21:48:57,296] Trial 3 finished with value: 0.8931736707986991 and parameters: {'subsample': 0.7999557893659686, 'n_estimators': 333, 'eta': 0.030397070464868767, 'min_child_weight': 4.250658165420043, 'colsample_bytree': 0.3665610312194021}. Best is trial 3 with value: 0.8931736707986991.
[I 2023-11-25 21:48:58,258] Trial 1 finished with value: 0.8947465086374665 and parameters: {'subsample': 0.6413966815745723, 'n_estimators': 343, 'eta': 0.04655605382228759, 'min_child_weight': 3.109211884749828, 'colsample_bytree': 0.36137873220597416}. Best is trial 1 with value: 0.8947465086374665.
[I 2023-11-25 21:49:03,490] Trial 0 finished with value: 0.8973526171046455 and parameters: {'subsample': 0.7293430319573855, 'n_estimators': 460, 'eta': 0.08013068267306661, 'min_child_weight': 3.7053664646375415, 'colsample_bytree': 0.20975009364306813}. Best is trial 0 with value: 0.8973526171046455.
[I 2023-11-25 21:49:04,619] Trial 2 finished with value: 0.8968773568271413 and paramet

[I 2023-11-25 21:49:53,715] A new study created in memory with name: no-name-6f6f2b99-3e48-46ee-a147-d15ab8d50733


...

[I 2023-11-25 21:50:45,619] Trial 3 finished with value: 0.8571098564929471 and parameters: {'num_iterations': 402, 'colsample_bytree': 0.121263621904283, 'num_leaves': 18, 'learning_rate': 0.47711244831471655, 'boosting_type': 'goss', 'max_bin': 2535}. Best is trial 3 with value: 0.8571098564929471.
[I 2023-11-25 21:51:09,838] Trial 2 finished with value: 0.8832706173496105 and parameters: {'num_iterations': 451, 'colsample_bytree': 0.4976801423461463, 'num_leaves': 17, 'learning_rate': 0.16814122123192368, 'boosting_type': 'goss', 'max_bin': 2309}. Best is trial 2 with value: 0.8832706173496105.
[I 2023-11-25 21:51:40,892] Trial 4 finished with value: 0.871078735744066 and parameters: {'num_iterations': 399, 'colsample_bytree': 0.342550952299299, 'num_leaves': 15, 'learning_rate': 0.47770374566332563, 'boosting_type': 'gbdt', 'max_bin': 3374}. Best is trial 2 with value: 0.8832706173496105.
[I 2023-11-25 21:52:00,208] Trial 5 finished with value: 0.8457946122635036 and parameters: {'

[I 2023-11-25 21:55:39,786] A new study created in memory with name: no-name-02fdf406-fb30-4760-bb67-14660c9012b3


Result for LGBMClassifier: 0.8898742940894099


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-11-25 22:04:26,739] Trial 1 finished with value: 0.8742238899428554 and parameters: {'iterations': 703, 'learning_rate': 0.005077775093562668, 'auto_class_weights': 'SqrtBalanced', 'depth': 3, 'nan_mode': 'Min'}. Best is trial 1 with value: 0.8742238899428554.
[I 2023-11-25 22:04:56,391] Trial 3 finished with value: 0.8787708316843983 and parameters: {'iterations': 607, 'learning_rate': 0.009919943840540967, 'auto_class_weights': 'None', 'depth': 4, 'nan_mode': 'Max'}. Best is trial 3 with value: 0.8787708316843983.
[I 2023-11-25 22:08:42,642] Trial 2 finished with value: 0.8828138195266848 and parameters: {'iterations': 748, 'learning_rate': 0.009727177000854624, 'auto_class_weights': 'Balanced', 'depth': 4, 'nan_mode': 'Max'}. Best is trial 2 with value: 0.8828138195266848.
[I 2023-11-25 22:12:36,141] Trial 5 finished with value: 0.8722977967832024 and parameters: {'iterations': 584, 'learning_rate': 0.0050403366820583875, 'auto_class_weights': 'SqrtBalanced', 'depth': 3, 'na

In [107]:
trained_models = []  # Список для хранения обученных моделей

for result in optimization_results:
    for model in MLA:
        if model.__class__.__name__ == result['model_name']:
            valid_params = model.get_params()
            filtered_params = {k: v for k, v in result['best_trial_params'].items() if k in valid_params}
            
            # Создаем новый экземпляр модели с лучшими гиперпараметрами
            if result['model_name'] == "CatBoostClassifier":
                model_instance = CatBoostClassifier(**filtered_params)
            else:
                model_instance = model.set_params(**filtered_params)
            
            trained_models.append(model_instance)
            

In [108]:
trained_models

[XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.2314755702624563, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=3.934527731868575, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=474,
               n_jobs=None, num_parallel_tree=None, random_state=None, ...),
 LGBMClassifier(boosting_type='dart', colsample_bytree=0.376215733173456,
                device_type='gpu', learning_rate=0.29320263065359176,
                num_leaves=13),

In [151]:
from sklearn.metrics import roc_auc_score

def print_roca_auc(clf, X, y, label):
    a = clf.predict(X)
    
    print(label + ' AUC-ROC  = ' + str(roc_auc_score(y, a)))

In [150]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict_proba(valid)[:,1]
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict_proba(X)[:, 1]
        
        a = self.ens_model.predict_proba(X_meta)[:, 1]
        
        return (a)

In [145]:
train_X, test_X, train_y, test_y = train_test_split(x_train[x_cols], x_train[y_cols], train_size=0.8, 
                                                    random_state=1999, stratify=x_train[y_cols])
ens_model = ensemble.ExtraTreesClassifier(n_jobs=n_jobs, verbose = 0)
s1 = DjStacking(trained_models, ens_model)
s1.fit(train_X, train_y)

# model_name = f'{run.name}_model.mod'
# bstr = s1.booster_
# bstr.save_model(model_name)
# config = s1.get_params()
# model_art = wandb.Artifact(name=model_name, type='model', metadata=config)
# model_art.add_file(model_name)
# run.log_artifact(model_art)

# run.summary["best_score"] = bstr.best_score
# run.summary["best_iteration"] = bstr.best_iteration
# preds = model.predict(x_val)
# run.summary["f1_score"] = f1_score(y_val, preds, average='macro')
# run.summary["accuracy"] = accuracy_score(y_val, preds)

# model_name = f'{run.name}_model.mod'
# bstr = s2.booster_
# bstr.save_model(model_name)
# config = s2.get_params()
# model_art = wandb.Artifact(name=model_name, type='model', metadata=config)
# model_art.add_file(model_name)
# run.log_artifact(model_art)

# run.summary["best_score"] = bstr.best_score
# run.summary["best_iteration"] = bstr.best_iteration
# preds = model.predict(x_val)
# run.summary["f1_score"] = f1_score(y_val, preds, average='macro')
# run.summary["accuracy"] = accuracy_score(y_val, preds)

# run.finish()

Learning rate set to 0.098539
0:	learn: 0.5648291	total: 86.9ms	remaining: 1m 26s
1:	learn: 0.4694617	total: 166ms	remaining: 1m 22s
2:	learn: 0.3941851	total: 255ms	remaining: 1m 24s
3:	learn: 0.3453618	total: 330ms	remaining: 1m 22s
4:	learn: 0.3057702	total: 406ms	remaining: 1m 20s
5:	learn: 0.2776494	total: 472ms	remaining: 1m 18s
6:	learn: 0.2557253	total: 557ms	remaining: 1m 18s
7:	learn: 0.2399213	total: 626ms	remaining: 1m 17s
8:	learn: 0.2272521	total: 697ms	remaining: 1m 16s
9:	learn: 0.2171021	total: 771ms	remaining: 1m 16s
10:	learn: 0.2108773	total: 854ms	remaining: 1m 16s
11:	learn: 0.2060472	total: 931ms	remaining: 1m 16s
12:	learn: 0.2006249	total: 1.01s	remaining: 1m 16s
13:	learn: 0.1964120	total: 1.1s	remaining: 1m 17s
14:	learn: 0.1938010	total: 1.18s	remaining: 1m 17s
15:	learn: 0.1912040	total: 1.26s	remaining: 1m 17s
16:	learn: 0.1892147	total: 1.33s	remaining: 1m 16s
17:	learn: 0.1879426	total: 1.4s	remaining: 1m 16s
18:	learn: 0.1864086	total: 1.48s	remaining: 

DjStacking(ens_model=ExtraTreesClassifier(n_jobs=-1),
           models=[XGBClassifier(base_score=None, booster=None, callbacks=None,
                                 colsample_bylevel=None, colsample_bynode=None,
                                 colsample_bytree=0.2314755702624563,
                                 device=None, early_stopping_rounds=None,
                                 enable_categorical=False, eval_metric=None,
                                 feature_types=None, gamma=None,
                                 grow_policy=None, importance_type=None,
                                 i...
                                 min_child_weight=3.934527731868575,
                                 missing=nan, monotone_constraints=None,
                                 multi_strategy=None, n_estimators=474,
                                 n_jobs=None, num_parallel_tree=None,
                                 random_state=None, ...),
                   LGBMClassifier(boosting_type='dart',
                                  colsample_bytree=0.376215733173456,
                                  device_type='gpu',
                                  learning_rate=0.29320263065359176,
                                  num_leaves=13),
                   <catboost.core.CatBoostClassifier object at 0x7d055ba9bb20>])

In [152]:
print_roca_auc(s1, test_X, test_y, '1-stacking')

AttributeError: 'DjStacking' object has no attribute 'predict'

In [143]:
s1.predict(test_df)

ValueError: train and valid dataset categorical_feature do not match.

In [142]:
test_df = test_df.astype({'channel_code': 'category', 'branch_code': 'category'})

In [ ]:
s1 = DjStacking(trained_models, ens_model)
a = 0
e = []
for t in range(2, 11):
    s1.fit(train_X, train_y, p=-1, cv=t, err=0.00)
    a = s1.predict(test_X, train_y)
    auc = roc_auc_score(test_y, a)
    print (auc)
    e.append(auc)

# Pipeline for ensemble